In [32]:
import random
import re
from collections import Counter
import time
import pickle
import numpy as np

In [2]:
ALPHABET_LIST = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = set(ALPHABET_LIST)

def load_words():
    with open('words.txt') as f:
        valid_words = set(f.read().split())

    return valid_words

def filter_words(words, verbose=1):
    output = words.copy()
    for word in words:
        # remove triple letters
        for index, letter in enumerate(word[2:]):
            if (word[index] == word[index + 1] == letter) or letter not in ALPHABET:
                if verbose > 0:
                    print(f'Removing \'{word}\'')
                output.remove(word)
                break
    
    return output

def check_words(words, *word_list):
    output = []
    for word in word_list:
        output.append(word in words)
    
    if len(output) == 1:
        return output[0]
    
    return output

def get_random(words, count):
    for i in range(count):
        return random.sample(list(words), count)

WORDS = filter_words(load_words(), verbose=0)
WORDS = WORDS.union({'joyousnesses'})

In [3]:
print(check_words(WORDS, 'hexadecimal', 'elucidate', 'perfunctory', 'fhqwhgads', 'eef', 'gnarf'))
print(get_random(WORDS, 10))

[True, True, True, False, False, False]
['alveolated', 'harpoon', 'signance', 'axiferous', 'pelean', 'cardinalis', 'corin', 'nephrectomy', 'drubbings', 'misprise']


In [39]:
class Chunk:
    def __init__(self, text):
        self.text = text
        
    def __eq__(self, other):
        return self.text == other.text
    
class Word(Chunk):
    def __init__(self, text):
        super().__init__(text)
        self.reveal = ['_'] * len(text)
        self.not_present = []
        self.remaining = len(text)

    def reveal_guess(self, guess):
        success = False
        for index, letter in enumerate(self.text):
            if letter == guess:
                self.reveal[index] = letter
                self.remaining -= 1
                success = True
                
        if not success:
            self.not_present.append(guess)

        return success
    
    def get_matches(self, words):
        words = list(filter(lambda word: not any(letter in self.not_present for letter in word), words))
        pattern = re.compile('^' + ''.join(map(lambda let: '.' if let == '_' else let, self.reveal)) + '$')
        return list(filter(pattern.match, words))
    
    def full_reveal(self):
        self.reveal = list(self.text)
        self.remaining = 0
    
    def solved(self):
        return self.remaining <= 0
    
    def __str__(self):
        return ''.join(self.reveal)
    
class Space(Chunk):
    def __init__(self, text):
        super().__init__(text)
    
    def reveal_guess(self, letter):
        return False
    
    def full_reveal(self):
        pass
    
    def solved(self):
        return True
    
    def __str__(self):
        return self.text
    
class Solution:
    def __init__(self, valid_words, solution):
        self.text, self.solution = Solution.parse_solution(valid_words, solution)
        
    def parse_solution(words, solution):
        if '_' in solution:
            raise ValueError('No underscores allowed.')

        solution = solution.strip().lower()
        found_words = re.split(PATTERN, solution)

        solution_words = []
        solution_revealed = []
        for word in found_words:
            if word.isnumeric():
                solution_words.append(Space(word))

            elif word.isalnum():
                if check_words(words, word):
                    solution_words.append(Word(word))
                else:
                    raise ValueError('Solution includes non-word.')

            else:
                solution_words.append(Space(word))

        # check that there is at least one word
        for word in solution_words:
            if isinstance(word, Word):
                return solution, solution_words

        raise ValueError('No words found in solution.')
    
    def print_reveal(self):
        out = ''
        for chunk in self.solution:
            out = out + chunk
        print(out)
    
    def full_reveal(self):
        for chunk in self.solution:
            chunk.full_reveal()
            
    def valid_guess(self, guess):
        if not isinstance(guess, Solution):
            return False
        
        lmbda = lambda chunk: len(chunk.text) if isinstance(chunk, Word) else 0
        s1 = list(map(lmbda, self.words_only()))
        s2 = list(map(lmbda, guess.words_only()))
        return s1 == s2
    
    def words_only(self):
        return list(filter(lambda chunk: isinstance(chunk, Word), self.solution))
    
    def __iter__(self):
        return iter(self.solution)
    
    def __eq__(self, other):
        if not isinstance(other, Solution):
            return False
        
        s1 = self.words_only()
        s2 = other.words_only()
        return s1 == s2
    
    def __str__(self):
        out = ''
        for chunk in self.solution:
            out = out + chunk.text
        return out

In [40]:
class Hangman:
    # solution is a list of chunks
    def __init__(self, valid_words, solution):
        self.valid_words = valid_words
        self.solution = Solution(self.valid_words, solution)
        self.failed_guesses = []
        self.guessed = []
        self.guesses = 0
        
    def try_solution(self, guess):
        s = Solution(self.valid_words, guess)
        if not self.solution.valid_guess(s):
            raise ValueError('Guess must have same the number/length of words as solution.')

        as_string = s.__str__()
        if as_string in self.guessed:
            raise ValueError('Attempted solution already guessed.')

        self.guessed.append(as_string)
        self.guesses += 1

        if s == self.solution:
            self.solution.full_reveal()
        else:
            self.failed_guesses.append(as_string)
    
    def reveal_guess(self, guess):
        guess = guess.strip().lower()
        
        if len(guess) > 1:
            self.try_solution(guess)
            return
        
        if len(guess) < 1:
            raise ValueError('Guess must be at least one letter.')
        if not guess.isalpha():
            raise ValueError('Guess must be alphabetic.')
        if guess in self.guessed:
            raise ValueError('Letter already guessed.')
        
        success = False
        for chunk in self.solution:
            success = chunk.reveal_guess(guess) or success
            
        self.guesses += 1
        self.guessed.append(guess)
        
        if not success:
            self.failed_guesses.append(guess)
        
        return success
            
    def solved(self):
        return all(chunk.solved() for chunk in self.solution)
    
    def print_guessed(self):
        print(f'Guessed: {self.failed_guesses}')
        
    def print_reveal(self):
        for chunk in self.solution:
            print(chunk, end='')
        print()
    
    def print_win(self):
        g = 'guess' if self.guesses == 1 else 'guesses'
        print(f'Congratulations! You solved the puzzle in {self.guesses} {g}.')
        
    def print_win_basic(self):
        g = 'guess' if self.guesses == 1 else 'guesses'
        print(f'{self.guesses} {g}')

In [87]:
def get_most_common_letters(words, verbose=0):
    letter_counts = {letter: 0 for letter in ALPHABET}
    for word in words:
        counts = Counter(word)
        for key, ct in counts.items():
            letter_counts[key] += ct
            
    result = list(sorted(ALPHABET, key=lambda key: letter_counts[key], reverse=True))
    
    if verbose > 0:
        for letter in result:
            print(f'{letter}: {letter_counts[letter]}')
    
    return result

MOST_COMMON_LETTERS = get_most_common_letters(WORDS, verbose=1)

e: 376426
i: 312966
a: 295775
o: 251582
n: 251425
s: 250257
r: 246130
t: 230880
l: 194895
c: 152975
u: 131493
p: 113637
d: 113178
m: 105200
h: 92361
g: 82622
y: 70580
b: 63930
f: 39238
v: 33073
k: 26812
w: 22405
z: 14755
x: 10486
q: 5883
j: 5457


In [104]:
class Guesser:
    def __init__(self, words):
        self.words = words
        
    def init(self, game):
        self.game = game
    
    def generate_guess(self):
        raise NotImplementedException()
        
    def reset(self):
        pass
        
class InputGuesser(Guesser):
    def generate_guess(self):
        return input('Input a guess: ')

class RandomGuesser(Guesser):
    def generate_guess(self):
        guess = random.choice(ALPHABET_LIST)
        while guess in self.game.guessed:
            guess = random.choice(ALPHABET_LIST)
        return guess

class OrderedGuesser(Guesser):
    def __init__(self, words):
        super().__init__(words)
        self.index = 0
    
    def generate_guess(self):
        guess = MOST_COMMON_LETTERS[self.index]
        self.index += 1
        return guess
    
    def reset(self):
        self.index = 0

class SmartGuesser(Guesser):
    def __init__(self, words):
        super().__init__(words)
        self.possible_words = self.words
        
    def generate_guess(self):
        possible_words = []
        for sol_word in self.game.solution.words_only():
            possible_words.extend(sol_word.get_matches(self.possible_words))
        
        self.possible_words = possible_words
        
        if len(self.possible_words) == 1:
            return self.possible_words[0]
        
        # find the most common unused letter among possible words
        i = 0
        most_common = get_most_common_letters(self.possible_words)
        guess = most_common[i]
        while guess in self.game.guessed:
            i += 1
            guess = most_common[i]
        
        return guess
    
    def reset(self):
        self.possible_words = self.words

class DeepGuesser(Guesser):
    def __init__(self, words, depth=1, randombound=100):
        super().__init__(words)
        self.depth = depth
        self.randombound = randombound
        self.possible_words = self.words
    
    def generate_guess(self):
        possible_words = []
        for sol_word in self.game.solution.words_only():
            possible_words.extend(sol_word.get_matches(self.possible_words))
        
        self.possible_words = possible_words
        
        if len(self.possible_words) == 1:
            return self.possible_words[0]
        
        # find the letter which decreases the number of possible words the most on average
        scores = self.score_guesses(self.possible_words, self.depth)
#         print(list(map(lambda i: (ALPHABET_LIST[i], scores[i]), range(len(ALPHABET_LIST)))))
        best = sorted(range(len(ALPHABET)), key=lambda i: scores[i])
    
        i = 0
        guess = ALPHABET_LIST[best[i]]
        while guess in self.game.guessed:
            i += 1
            guess = ALPHABET_LIST[best[i]]
        
        return guess
    
    def score_guesses(self, words, depth, guessed=None):
        if guessed is None:
            guessed = self.game.guessed
        
        
        if len(words) > self.randombound:
            words = random.sample(words, self.randombound)

        scores = []
        for letter in ALPHABET_LIST:
            if letter in guessed:
                scores.append(len(words))
                continue

            updated_guessed = guessed + [letter]
            possibilities = self.generate_possible_words(words, depth - 1, guessed=updated_guessed)
            
            # score is average number of possibilities left over
            avg = sum(len(possible_set) for possible_set in possibilities)/len(possibilities)
#             if avg < 2:
#                 print(f'possibilities for {letter}: {possibilities}')
            scores.append(avg)
        
        return scores
    
    def generate_possible_words(self, words, depth, guessed=None):
        if guessed is None:
            guessed = self.game.guessed
            
        possibilities = []
            
        for solution_word in words:
            s = Word(solution_word)
            for guess in guessed:
                s.reveal_guess(guess)
            possibilities.append(s.get_matches(words))
            
#             if len(possibilities[-1]) < 3:
#                 print(f'solution_word={s}={solution_word}, possibilities={possibilities[-1]}')
        
        if depth <= 0:
            return possibilities
        
        return possibilities
    
    def reset(self):
        self.possible_words = self.words

In [97]:
class Generator:
    def generate_word(self):
        raise NotImplementedError()
        
    def reset(self):
        raise NotImplementedError()

class InputGenerator(Generator):
    def generate_word(self):
        return input('Input a solution: ')
    
    def reset(self):
        pass
    
class SingleWordRandomGenerator(Generator):
    def __init__(self, words, seed=None):
        self.words = words
        
        if seed is None:
            seed = int(time.time())
        self.seed = seed
        random.seed(self.seed)
        
    def generate_word(self):
        return random.choice(list(self.words))
    
    def reset(self):
        random.seed(self.seed)

In [98]:
PATTERN = re.compile('(\W+)')

def play_hangman(words, generator, guesser, verbose=3, iterations=1):
    generator.reset()
    guess_counts = {}
    
    for i in range(iterations):
        guesser.reset()
        game = None
        
        while game is None:
            try:
                game = Hangman(words, generator.generate_word())
                guesser.init(game)
            except ValueError as e:
                if verbose > 2:
                    print(f'Error: {e}')

        while not game.solved():
            if verbose > 1:
                game.print_reveal()
                game.print_guessed()
                print()

            guessing = True
            while guessing:
                try:
                    game.reveal_guess(guesser.generate_guess())
                    guessing = False
                except ValueError as e:
                    if verbose > 2:
                        print(f'Error: {e}')
                        
        if verbose > 1:
            game.print_reveal()
            game.print_guessed()
            game.print_win()
            
        elif verbose > 0:
            game.print_reveal()
            game.print_win_basic()
            
        guess_counts[game.solution.text] = game.guesses
        
    return guess_counts

guess_counts = play_hangman(WORDS, InputGenerator(), InputGuesser(WORDS))

Input a solution: 
Error: No words found in solution.
Input a solution: a
_
Guessed: []

Input a guess: a
a
Guessed: []
Congratulations! You solved the puzzle in 1 guess.


In [99]:
counts_random, counts_ordered, counts_smart, counts_deep = {}, {}, {}, {}

try:
    with open('random_guesses.pkl', 'rb') as f:
            counts_random = pickle.load(f)
    with open('ordered_guesses.pkl', 'rb') as f:
        counts_ordered = pickle.load(f)
    with open('smart_guesses.pkl', 'rb') as f:
        counts_smart = pickle.load(f)
    with open('deep_guesses.pkl', 'rb') as f:
        counts_deep = pickle.load(f)
except:
    pass

In [105]:
generator = SingleWordRandomGenerator(WORDS, seed=42)
def union(d1, d2):
    for key, val in d2.items():
        if key not in d1:
            d1[key] = val

union(counts_random, play_hangman(WORDS, generator, RandomGuesser(WORDS), verbose=0, iterations=500))
union(counts_ordered, play_hangman(WORDS, generator, OrderedGuesser(WORDS), verbose=0, iterations=500))
union(counts_smart, play_hangman(WORDS, generator, SmartGuesser(WORDS), verbose=0, iterations=500))
union(counts_deep, play_hangman(WORDS, generator, DeepGuesser(WORDS, depth=1), verbose=0, iterations=500))

In [106]:
with open('random_guesses.pkl', 'wb') as f:
    pickle.dump(counts_random, f)
with open('ordered_guesses.pkl', 'wb') as f:
    pickle.dump(counts_ordered, f)
with open('smart_guesses.pkl', 'wb') as f:
    pickle.dump(counts_smart, f)
with open('deep_guesses.pkl', 'wb') as f:
    pickle.dump(counts_deep, f)

In [110]:
def print_data(counts):
    counts = list(counts.values())
    print(f'mean: {np.mean(counts)}, std: {np.round(np.std(counts), 3)}')

print_data(counts_random)
print_data(counts_ordered)
print_data(counts_smart)
print_data(counts_deep)

mean: 23.506, std: 2.624
mean: 17.284, std: 3.714
mean: 8.054, std: 2.497
mean: 7.298, std: 2.509
